In [1]:
    !pip install pybliometrics

In [2]:
# Importing the required libraries.
import csv, re, pandas as pd, numpy as np
from datetime import datetime
from pybliometrics.scopus import config, AbstractRetrieval
from pybliometrics.scopus.utils import create_config
from pybliometrics.scopus.exception import Scopus404Error, Scopus429Error, Scopus500Error
from urllib.error import HTTPError
from urllib3.exceptions import ConnectionError, NewConnectionError

In [3]:
# Setting the Scopus API Key.
_keys = [">>>f8c74df6ef01dd94ebd36c932a731461<<<"]
config["Authentication"]["APIKey"] = _keys.pop()

In [16]:
# Getting the list of manuscripts' EID.
list_eids_documents = pd.read_csv("C:/Users/xkulll/Desktop/hh_filer/1401_part_eid.csv", index_col=False, squeeze=True)

C:\Users\xkulll\AppData\Local\Temp\ipykernel_21632\1516394431.py:2: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  list_eids_documents = pd.read_csv("C:/Users/xkulll/Desktop/hh_filer/1401_part_eid.csv", index_col=False, squeeze=True)


In [17]:
# Printing the number of records.
print("Number of records: {}.".format(list_eids_documents.size))

Number of records: 23.


In [33]:
# Function to collect manuscripts' data.
def collect_data_manuscripts(list_eids_documents):
    data = []
    for key in list_eids_documents:
        record = {}
        error = True
        while error:
            try:
                paper = AbstractRetrieval(key, id_type="eid", view="FULL", refresh=True)
                error = False
                # Basic Attributes.
                record["id"] = paper.identifier
                record["doi"] = paper.doi
                record["eid"] = paper.eid
                record["title"] = paper.title
                record["description"] = paper.description
                record["publication_date"] = datetime.strptime(paper.coverDate, "%Y-%m-%d").date() \
                                            if paper.coverDate else None
                record["citation_num"] = paper.citedby_count
                record["language"] = paper.language
                record["production_type"] = paper.aggregationType
                record["source_type"] = paper.srctype
                record["auth_keywords"] = tuple(paper.authkeywords) if paper.authkeywords else None
                record["index_terms"] = tuple(paper.idxterms) if paper.idxterms else None
               

                # Journals data.
                record["source"] = paper.publicationName
                record["publisher"] = paper.publisher

                # Affiliation.
                record["affiliations"] = tuple(
                    [{"id": affil.id if affil and affil.id else None,
                      "affiliation": affil.name if affil and affil.name else None,
                      "country": affil.country if affil and affil.country else None}
                     for affil in paper.affiliation]) if paper.affiliation else None

                # Subject Areas.
                record["subject_areas"] = tuple([area.area for area in paper.subject_areas]) \
                                        if paper.subject_areas else None
                record["ajsc_codes"] = tuple([code.code for code in paper.subject_areas]) \
                                        if paper.subject_areas else None
             
                # Authors.
                record["authors"] = tuple(
                    [{"id": author.auid if author and author.auid else None,
                      "name": "{} {}".format(author.given_name, author.surname) \
                                if author and author.given_name and author.surname else
                            "{}".format(author.given_name) if author and author.given_name \
                                and not author.surname else \
                            "{}".format(author.surname) if author and author.surname \
                                and not author.given_name else None}
                     for author in paper.authors]) if paper.authors else None

                record["author_affil"] = tuple(
                    [{"id": author.auid if author and author.auid else None,
                      "name": "{} {}".format(author.given_name, author.surname) \
                                if author and author.given_name and author.surname else \
                            "{}".format(author.given_name) if author and author.given_name \
                                and not author.surname else \
                            "{}".format(author.surname) if author and author.surname \
                                and not author.given_name else None,
                      "affil_id": author.affiliation_id if author and author.affiliation_id else None,
                      "affiliation": author.organization if author and author.organization else None,
                      "country": author.country if author and author.country else None}
                     for author in paper.authorgroup]) if paper.authorgroup else None
               

            except (Scopus404Error, Scopus500Error, HTTPError) as e:
                record["id"] = key
                print(key)
                error = False
            except Scopus429Error as e:
                # Removing the last item in _keys to assign it as new API key.
                config["Authentication"]["APIKey"] = _keys.pop()
                if len(_keys) == 0:
                    raise e
            except (ConnectionError, NewConnectionError, Exception) as e:
                record["id"] = key
                print(key)
                error = False
        data.append(record)
    return data

In [34]:
# Collecting the data.
data = collect_data_manuscripts(list_eids_documents)    

In [35]:
# Exporting the data to CSV file.
pd.DataFrame(data).to_csv("C:/Users/xkulll/Desktop/hh_filer/scopus_part2_raw.csv", index=False, quoting=csv.QUOTE_ALL)